In [2]:
from pathlib import Path
from retinaface import RetinaFace
from tensorflow import keras
from keras_vggface.utils import preprocess_input
import numpy as np

In [24]:
import cv2

In [29]:
from keras_vggface.utils import preprocess_input

In [18]:
cwd = Path(os.path.abspath(''))
sys.path.append(str(cwd.parent.parent))
sys.path.append(str(cwd.parent.parent/"recognition"))

In [19]:
sys.path

['/Users/kmielcar/Documents/projects/faces_classification/classification/vgg',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload',
 '',
 '/Users/kmielcar/Documents/projects/faces_classification/venv/lib/python3.8/site-packages',
 PosixPath('/Users/kmielcar/Documents/projects/faces_classification/classification'),
 PosixPath('/Users/kmielcar/Documents/projects/faces_classification'),
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification/recognition']

In [20]:
from crop import crop

In [56]:
import importlib

In [60]:
import common

In [ ]:
importlib.reload(common)

In [62]:
from common import EXPECTED_SIZE, GIRLS

In [21]:
test_dir = Path("../../images/test/Grafiti/")
output_dir = Path("../../images/results/")

In [22]:
image_paths = list(test_dir.glob("*jpg"))

In [31]:
model = keras.models.load_model('trained_vgg_on_girls/')

In [77]:
def process_img(img_path:Path):
    out_path = output_dir / img_path.name
    
    if out_path.exists():
        return

    detected_faces_desc = RetinaFace.detect_faces(str(img_path))
    
    if type(detected_faces_desc) != dict:
        return

    faces = detected_faces_desc.values()
    if not faces:
        return
    
    print(img_path)

    img_bgr = cv2.imread(str(img_path))
    
    for face_details in faces:
        face_roi = face_details['facial_area']

        cropped_face_img = crop(img_bgr, *face_roi)
        
        cropped_face_img = cropped_face_img.astype('float32')
        sample = np.expand_dims(cropped_face_img, axis=0)
        sample = preprocess_input(sample, version=2)
        
        pred = model.predict(sample)
        max_val_ind = np.argmax(pred[0])
        proba = pred[0][max_val_ind]
        
        text = "{}: {:.2f}%".format(GIRLS[max_val_ind], proba * 100)
        left, top, right, bottom = face_roi
        y = top - 10 if top - 10 > 10 else top + 10
        cv2.rectangle(img_bgr, (left, top), (right, bottom), (0, 255, 0), 2)
        marked_img = cv2.putText(img_bgr, text, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
        
        
        cv2.imwrite(str(out_path), marked_img)

In [78]:
for img_path in image_paths:
    process_img(img_path)

../../images/test/Grafiti/1657797459985.jpg
1/1 [==============================] - 0s 60ms/step
../../images/test/Grafiti/1657797460123.jpg
1/1 [==============================] - 0s 65ms/step
../../images/test/Grafiti/1657797459831.jpg
1/1 [==============================] - 0s 58ms/step
